## 1 Data Loading
First we load in the raw data as a csv file (which is the file you directly get from MassHunter).
We also directly check if the s/n ratio is above 3 for all samples in the calibration line. 

In [12]:
%run src/functions_quantitative2.ipynb
rawdata = load_and_label_pfas_csv("rawdata/Results_Test_SPE50ml_16-12-2025.csv")
sn_results = check_calibration_sn_ratios(rawdata)
# Print formatted results
print_sn_check_results(sn_results)

Detected delimiter: ';'
['13C4-PFBA', '13C3-PFBA', 'PFBA', '13C4-PFBA', '13C5-PFPeA', '13C2-PFHxA', 'PFPeA', '13C5-PFPeA', '4:2 FTS', '13C2-4:2FTS', '13C2-4:2FTS', '18O2-PFHxS', '13C5-PFHxA', '13C2-PFHxA', 'PFHxA', '13C5-PFHxA', '13C3-PFBS', '18O2-PFHxS', 'PFBS', '13C3-PFBS', 'HFPO-DA', '13C3-HFPO-DA', '13C3-HFPO-DA', '13C2-PFHxA', '13C4-PFHpA', '13C2-PFHxA', 'PFPHpA', '13C4-PFHpA', 'PFPeS', '13C3-PFHxS', 'ADONA', '13C3-HFPO-DA', '13C2-6:2FTS', '18O2-PFHxS', '6:2 FTS', '13C2-6:2FTS', 'PFOA', '13C8-PFOA', '13C8-PFOA', '13C4-PFOA', '13C3-PFHxS', '18O2-PFHxS', 'PFHxS', '13C3-PFHxS', '13C9-PFNA', '13C5-PFNA', 'PFNA', '13C9-PFNA', 'PFHpS', '13C8-PFOS', '8:2 FTS', '13C2-8:2FTS', '13C2-8:2FTS', '18O2-PFHxS', 'PFDA', '13C6-PFDA', '13C6-PFDA', '13C2-PFDA', 'NMeFOSAA', 'D3-NMeFOSAA', 'D3-NMeFOSAA', '13C4-PFOS', 'PFOS', '13C8-PFOS', '13C8-PFOS', '13C4-PFOS', 'NEtFOSAA', 'D5-NEtFOSAA', 'D5-NEtFOSAA', '13C4-PFOS', 'PFUnA', '13C7-PFUnA', '13C7-PFUnA', '13C2-PFDA', '9Cl-PF3ONS', '13C3-HFPO-DA', 'PFNS

## 2 Calibration line:  Average Response Factor, recovery and RSE for EIS compounds 
To analyze all instrument linearity metrics, we first need to calculate the EIS concentrations found in the samples. 
We do this by caclulating the concentrations based on the formula on page 44 of EPA method 1633. 
To achieve this, we first need to calculate the average response factor per compound (EPA method 1633 page 28). 
Afterwards, we check if the recovery of the compound falls within the accepted limits denoted by EPA method 1633 (Table 6). 
We also already compute the concentrations for the QC samples, these are analyzed further in section 3.2 

We compute the RSE based on  the formula for RSE based on EPA method 1633 (section 10.3.3.3, page 28, option 2). The functions are loaded in a different script. 
If the RSE of a single compound remains above 20%, this means something is wrong with the linearity of the LC-MS TQ and the calculated concentrations in section 3 are not trustworthy. 

### 2.1 EIS compounds
MAKE SURE YOU CHANGE THE MATRIX TYPE FOR ACCURATE RECOVERIES

If you notice here certain calibration levels are consistently bad, there's an option to exclude them for further analysis in the 2nd code block. 
If you don't want to exclude any levels type *excluded_cal_level = 'None'*

The 2nd code block prints detailed information on the RSE. You can view detailed results by removing *%%capture* in the code block, or alternatively just observe the printed summary. 

### QC Cal samples 
If there's any QC samples included that have a level identified (i.e. CS4), the script will also print the results for this. If these recoveries do not meet required recovery standards, please proceed with caution.

In [1]:
from expected_concs import expected_concs_EIS, expected_concs_target_analytes
from analogs_compounds import target_EIS_analogs, EIS_NIS_analogs
from solutions import calibration_solutions
from solutions import NIS_stock, EIS_stock
from allowed_recoveries import EIS_NIS_compound_recovery_aqueous,EIS_NIS_compound_recovery_solid, OPR_recovery_aqueous, OPR_recovery_solid, IPR_recovery_rsd_solid, IPR_recovery_rsd_aqueous

print(expected_concs_target_analytes)
matrix_type = "aqueous"  # or "solid

# Select appropriate recovery limits based on matrix type
if matrix_type.lower() == "aqueous":
    EIS_NIS_compound_recovery = EIS_NIS_compound_recovery_aqueous
    OPR_recovery = OPR_recovery_aqueous
    IPR_recovery_rsd = IPR_recovery_rsd_aqueous
elif matrix_type.lower() == "solid":
    EIS_NIS_compound_recovery = EIS_NIS_compound_recovery_solid
    OPR_recovery = OPR_recovery_solid
    IPR_recovery_rsd = IPR_recovery_rsd_solid
else:
    raise ValueError(f"Unknown matrix type: {matrix_type}. Use 'aqueous' or 'solid'")

#calculations
eis_RFS = calculate_average_RFs_EIS(rawdata, EIS_NIS_analogs, calibration_solutions)


Ws_cal = 0.25#ml
dilution_NIS_stock_cal = 40 #dilution cal
dilution_EIS_stock_cal = 40 #dilution cal
spiked_amount_cal = 0.05 #ml
Df_cal = 1 #dilution factor
spiked_amount = 0.05 #ml

#calibration line
conc_EIS_cal, conc_QC = calculate_conc_EIS_cal(rawdata, eis_RFS, dilution_NIS_stock_cal, Ws_cal,
                          EIS_NIS_analogs, Df_cal, NIS_stock, spiked_amount_cal)

calculated_recoveries_EIS_cal = calculate_recoveries(conc_EIS_cal, expected_concs_EIS)
calculated_recoveries_EIS_cal.to_csv("results/calculated_recoveries_EIS_cal.csv", index=False)
conc_EIS_cal, conc_QC = calculate_conc_EIS_cal(rawdata, eis_RFS, dilution_NIS_stock_cal, Ws_cal,
                          EIS_NIS_analogs, Df_cal, NIS_stock, spiked_amount_cal)
validation_results_cal = validate_recoveries(calculated_recoveries_EIS_cal, EIS_NIS_compound_recovery)
validation_results_cal= validate_recoveries(calculated_recoveries_EIS_cal, EIS_NIS_compound_recovery)
summary_cal = summarize_validation_results(validation_results_cal)
print("Validation Summary:")
for key, value in summary_cal['Overall'].items():
    print(f"{key}: {value}")
failed_recoveries_cal = get_failed_recoveries(validation_results_cal)
# QC calculations - only run if QC samples exist
if conc_QC is not None and not conc_QC.empty:
    calculated_recoveries_QC_cal = calculate_recoveries(conc_QC, expected_concs_EIS)
    calculated_recoveries_QC_cal.to_csv("results/calculated_recoveries_QC_cal.csv", index=False)
    validation_results_QC = validate_recoveries(calculated_recoveries_QC_cal, EIS_NIS_compound_recovery)



{'PFBA': [0.8, 2, 5, 10, 20, 50, 250], '4:2FTS': [0.8, 2, 5, 10, 20, 50, 250], '6:2FTS': [0.8, 2, 5, 10, 20, 50, 250], '8:2FTS': [0.8, 2, 5, 10, 20, 50, 250], 'HFPO-DA': [0.8, 2, 5, 10, 20, 50, 250], 'ADONA': [0.8, 2, 5, 10, 20, 50, 250], '11Cl-PF3OUdS': [0.8, 2, 5, 10, 20, 50, 250], '9Cl-PF3ONS': [0.8, 2, 5, 10, 20, 50, 250], 'PFPeA': [0.4, 1, 2.5, 5, 10, 25, 125], 'NMeFOSE': [2, 5, 12.5, 25, 50, 125, 625], 'NEtFOSE': [2, 5, 12.5, 25, 50, 125, 625], 'PFHxA': [0.2, 0.5, 1.25, 2.5, 5, 12.5, 62.5], 'PFHpA': [0.2, 0.5, 1.25, 2.5, 5, 12.5, 62.5], 'PFOA': [0.2, 0.5, 1.25, 2.5, 5, 12.5, 62.5], 'PFNA': [0.2, 0.5, 1.25, 2.5, 5, 12.5, 62.5], 'PFDA': [0.2, 0.5, 1.25, 2.5, 5, 12.5, 62.5], 'PFUnA': [0.2, 0.5, 1.25, 2.5, 5, 12.5, 62.5], 'PFDoA': [0.2, 0.5, 1.25, 2.5, 5, 12.5, 62.5], 'PFTrDA': [0.2, 0.5, 1.25, 2.5, 5, 12.5, 62.5], 'PFTeDA': [0.2, 0.5, 1.25, 2.5, 5, 12.5, 62.5], 'PFBS': [0.2, 0.5, 1.25, 2.5, 5, 12.5, 62.5], 'PFPeS': [0.2, 0.5, 1.25, 2.5, 5, 12.5, 62.5], 'PFHxS': [0.2, 0.5, 1.25, 2.5,

NameError: name 'calculate_average_RFs_EIS' is not defined

In [14]:
#%%capture
# RSE EIS Calibration line
excluded_cal_level = 'L6'
rse_results_cal_EIS = calculate_eis_rse_modified(conc_EIS_cal, expected_concs_EIS, p=2, exclude_levels = excluded_cal_level)


EIS compounds found: 24
EIS compounds with expected concentrations: 24
Excluding calibration levels: ['L6']
Excluded 1 samples from levels ['L6']
Number of Cal samples used for analysis: 6
Calibration levels used: ['L1', 'L2', 'L3', 'L4', 'L5', 'L7']

D3-NMeFOSAA (p=2)
Point	Expected	Measured	Difference	(Diff)²/xi²	(Diff)²/xi²/(n-p)
--------------------------------------------------------------------------------
CAL 1	5		5.1154		0.1154		0.000532	0.000133
CAL 2	5		4.7499		-0.2501		0.002501	0.000625
CAL 3	5		4.4032		-0.5968		0.014247	0.003562
CAL 4	5		4.7095		-0.2905		0.003375	0.000844
CAL 5	5		4.9512		-0.0488		0.000095	0.000024
CAL 6	5		6.0903		1.0903		0.047554	0.011889
--------------------------------------------------------------------------------
Sum of relative squared errors: 0.017076
RSE = 100 * √(0.017076) = 13.068%
D3-NMeFOSAA: Using 6 data points for RSE calculation

13C2-8:2FTS (p=2)
Point	Expected	Measured	Difference	(Diff)²/xi²	(Diff)²/xi²/(n-p)
-----------------------------

In [15]:
df_rse_eis=df_rse_results(rse_results_cal_EIS)
display(df_rse_eis)

,Compound,RSE_Percent,Calibration_Points,Relationship_Type,Parameters,Pass_Fail,Sum_Relative_Squared_Error
0,D3-NMeFOSAA,13.067585,6,linear,2,PASS,0.017076
1,13C2-8:2FTS,8.081244,6,linear,2,PASS,0.006531
2,13C2-4:2FTS,11.063130,6,linear,2,PASS,0.012239
3,13C2-PFTeDA,14.070429,6,linear,2,PASS,0.019798
4,D7-NMeFOSE,9.165878,6,linear,2,PASS,0.008401
5,13C8-PFOSA,4.305061,6,linear,2,PASS,0.001853
6,D5-NEtFOSA,7.816321,6,linear,2,PASS,0.006109
7,13C3-PFHxS,5.589111,6,linear,2,PASS,0.003124
8,13C5-PFPeA,2.805916,6,linear,2,PASS,0.000787
9,D3-NMeFOSA,6.082349,6,linear,2,PASS,0.003699


## 2.2 Target analytes 

### 2.2.1 Response Ratio (RR) and Response Factor (RF) for target ID compounds
Here we compute the response ratio for the ID compounds by using the formula from EPA 1633 on page 27 under section 10.3.3.2. For now we also fix some naming errors in the raw data for the target analytes. The only difference with the above calculation is that EIS analogs instead of direct pairs are used for the target EIS compounds. For ease, we call both RF.

- *Note 1*: NMeFoSA and NEtFoSa are calculated now but are not included in the PFAC30PAR ampoule we currently use in the lab, so these compounds cannot be in the calibration line. Use these results with care.
- *Note 2*:EPA 1633 mentions that PFTrDA recovery should improve by taking the average of the analog EIS compounds 13c2-PFTeDA and 13c2-PFDoa. Currently it only takes the EIS compound 13C2-PFTeDA as an analog.
- *Note 3*: EPA 1633 is capable of analyzing 40 compounds, our native standard solution only contains 30, so you will get errors that data for some compounds is not found. This is correct. If new ampoules are ordered for the other 10 compounds, everything needed for the calculation is there.
- *Note 4*: the FTS compounds only have 6 cal levels instead of 7.


In [16]:
from analogs_compounds import target_EIS_analogs

# Make a copy of rawdata to avoid modifying the original
rawdata_copy = rawdata.copy()
columns_to_rename = {}
for col in rawdata_copy.columns:
    new_col = col
    # Fix PFPHpA -> PFHpA
    if 'PFPHpA' in col:
        new_col = new_col.replace('PFPHpA', 'PFHpA')
    # Remove spaces from FTS compounds (e.g., "4:2 FTS" -> "4:2FTS")
    if ' FTS' in col:
        new_col = new_col.replace(' FTS', 'FTS')
    
    if new_col != col:
        columns_to_rename[col] = new_col

if columns_to_rename:
    rawdata_copy.rename(columns=columns_to_rename, inplace=True)

RF_target_analytes= calculate_average_RRs_targets(rawdata_copy, expected_concs_EIS, calibration_solutions, target_EIS_analogs)

Found 40 total target-IS pairs:
  NMeFOSE -> D7-NMeFOSE
  NEtFOSE -> D9-NEtFOSE
  PFBA -> 13C4-PFBA
  HFPO-DA -> 13C3-HFPO-DA
  PFPeA -> 13C5-PFPeA
  4:2FTS -> 13C2-4:2FTS
  6:2FTS -> 13C2-6:2FTS
  8:2FTS -> 13C2-8:2FTS
  NMeFOSAA -> D3-NMeFOSAA
  NEtFOSAA -> D5-NEtFOSAA
  PFHxA -> 13C5-PFHxA
  PFHpA -> 13C4-PFHpA
  PFOA -> 13C8-PFOA
  PFBS -> 13C3-PFBS
  PFHxS -> 13C3-PFHxS
  PFOS -> 13C8-PFOS
  PFOSA -> 13C8-PFOSA
  NMeFOSA -> D3-NMeFOSA
  NEtFOSA -> D5-NEtFOSA
  PFNA -> 13C9-PFNA
  PFDA -> 13C6-PFDA
  PFUnA -> 13C7-PFUnA
  PFDoA -> 13C2-PFDoA
  PFTeDA -> 13C2-PFTeDA
  PFTrDA -> 13C2-PFTeDA
  PFPeS -> 13C3-PFHxS
  PFHpS -> 13C8-PFOS
  PFNS -> 13C8-PFOS
  PFDS -> 13C8-PFOS
  PFDoS -> 13C8-PFOS
  ADONA -> 13C3-HFPO-DA
  PFMPA -> 13C5-PFPeA
  PFMBA -> 13C5-PFPeA
  NFDHA -> 13C5-PFHxA
  9Cl-PF3ONS -> 13C3-HFPO-DA
  11Cl-PF3OUdS -> 13C3-HFPO-DA
  PFEESA -> 13C5-PFHxA
  3:3FTCA -> 13C5-PFPeA
  5:3FTCA -> 13C5-PFHxA
  7:3FTCA -> 13C5-PFHxA


AVERAGE RESPONSE RATIOS FOR TARGET COMPOUNDS WITH

,Target_Compound,EIS_Isotope,Mass_Target (L4),Mass_EIS,Average_Response_Ratio,RSD_%,N_Levels,Levels_Used
0,NMeFOSE,D7-NMeFOSE,25.0,25.00,0.6971,4.91,7,"L1, L2, L3, L4, L5, L6, L7"
1,NEtFOSE,D9-NEtFOSE,25.0,25.00,0.6859,5.69,7,"L1, L2, L3, L4, L5, L6, L7"
2,PFBA,13C4-PFBA,10.0,10.00,0.6541,5.23,7,"L1, L2, L3, L4, L5, L6, L7"
3,HFPO-DA,13C3-HFPO-DA,10.0,10.00,0.7271,6.05,7,"L1, L2, L3, L4, L5, L6, L7"
4,PFPeA,13C5-PFPeA,5.0,5.00,0.6391,5.37,7,"L1, L2, L3, L4, L5, L6, L7"
5,4:2FTS,13C2-4:2FTS,10.0,5.00,2.5982,10.52,6,"L1, L2, L3, L4, L5, L6"
6,6:2FTS,13C2-6:2FTS,10.0,5.00,2.0273,12.52,6,"L1, L2, L3, L4, L5, L6"
7,8:2FTS,13C2-8:2FTS,10.0,5.00,1.8502,8.12,6,"L1, L2, L3, L4, L5, L6"
8,NMeFOSAA,D3-NMeFOSAA,2.5,5.00,0.6491,11.17,7,"L1, L2, L3, L4, L5, L6, L7"
9,NEtFOSAA,D5-NEtFOSAA,2.5,5.00,0.6140,13.59,7,"L1, L2, L3, L4, L5, L6, L7"


### 2.2.2 Recovery IPR and OPR for target analytes in calibration line 
Here we compare calculated recoveries for the target analytes in all calibration samples and compare them to the expected recoveries and maximum RSD for IPR (Table 5) 
* Note: OPR/LLOPR recovery is a little less strict. If needed for comparison exchange IPR_recovery_rsd_aqueous for OPR_recovery_aqueous. IPR considers mean recovery and RSD, OPR is per sample. Thus, you should not run the function mean_recoveries. 

In [17]:
from allowed_recoveries import IPR_recovery_rsd_aqueous, OPR_recovery_aqueous
from expected_concs import expected_concs_EIS, expected_concs_target_analytes
sampletype = "Cal"
conc_target_analytes_cal = calculate_conc_targets(rawdata_copy,
                           RF_target_analytes,
                           EIS_stock, dilution_EIS_stock_cal, spiked_amount_cal, Ws_cal,
                           Df_cal, sampletype, debug_compound=None )
calculated_recoveries_target_cal = calculate_recoveries(conc_target_analytes_cal, expected_concs_target_analytes)


sampletype2  = "QC" 


QC_levels = []  # Change this to [5], [3, 5], [1, 3, 5, 7], etc.

QC_target_analytes = calculate_conc_targets(
    rawdata_copy,
    RF_target_analytes,
    EIS_stock, dilution_EIS_stock_cal, spiked_amount_cal, Ws_cal,
    Df_cal, sampletype2, debug_compound=None
)

# Filter to the specified QC level(s)
qc_indices = [level - 1 for level in QC_levels]  # Convert levels to indices

# Create expected concentrations dictionary with only the specified levels
expected_concs_qc = {}
for compound, conc_list in expected_concs_target_analytes.items():
    expected_concs_qc[compound] = [conc_list[idx] for idx in qc_indices]

calculated_recoveries_qc = calculate_recoveries(
    QC_target_analytes, 
    expected_concs_qc
)
calculated_recoveries_QC = calculate_recoveries(QC_target_analytes, expected_concs_target_analytes)
# Save to CSV
calculated_recoveries_target_cal.to_csv('results/calculated_recoveries_target_cal.csv', index=False)
#calculated_recoveries_qc.to_csv('calculated_recoveries_qc.csv', index=False)
validation_results_target_cal= validate_recoveries(calculated_recoveries_target_cal, IPR_recovery_rsd)
summary_cal = summarize_validation_results(validation_results_target_cal)
print("Validation Summary:")
for key, value in summary_cal['Overall'].items():
    print(f"{key}: {value}")

# #for IPR, it is not necessarily failed if the below dataframe contains samples that do not pass. Rather, look at the output from mean_recoveries. 

# failed_recoveries_cal = get_failed_recoveries(validation_results_target_cal)
# #Only for IPR
# IPR_target_cal = mean_recoveries(calculated_recoveries_target_cal, IPR_recovery_rsd)
# display(IPR_target_cal)

,Name,Type,PFBA_Recovery_%,4:2FTS_Recovery_%,6:2FTS_Recovery_%,8:2FTS_Recovery_%,HFPO-DA_Recovery_%,ADONA_Recovery_%,11Cl-PF3OUdS_Recovery_%,9Cl-PF3ONS_Recovery_%,...,PFHxS_Recovery_%,PFHpS_Recovery_%,PFOS_Recovery_%,PFOSA_Recovery_%,NMeFOSA_Recovery_%,NEtFOSA_Recovery_%,NMeFOSAA_Recovery_%,NEtFOSAA_Recovery_%,PFNS_Recovery_%,PFDS_Recovery_%
1,CS1,Cal,103.247080,113.363186,73.282481,101.051677,106.537573,99.139414,95.869441,104.139325,...,94.832179,108.271910,121.194972,103.083082,112.725601,106.178123,115.767230,80.150812,91.574642,104.264785
2,CS2,Cal,106.453740,100.914141,100.297604,111.127163,105.601416,106.612078,101.665395,104.109081,...,109.618018,94.625806,110.835895,101.339557,104.418886,117.636223,91.811363,111.792583,101.080965,86.744858
3,CS3,Cal,100.454791,106.509381,104.543135,106.380606,102.355589,100.988172,99.385967,102.852799,...,103.250555,94.776443,104.384523,102.800747,99.544559,98.554534,100.195155,82.409122,108.911230,102.140873
4,CS4,Cal,95.321028,80.285307,105.125323,85.371606,92.473369,92.044061,89.026662,91.775010,...,96.513975,106.111031,89.004451,93.194055,95.353882,93.386880,86.570113,98.860569,98.991071,92.656801
5,CS5,Cal,90.196632,94.336779,103.938384,100.032778,90.347144,93.602481,92.835780,93.929662,...,92.987962,100.292058,85.668598,91.521215,89.870776,86.542541,100.971407,97.436271,99.638393,99.038019
6,CS6,Cal,104.544975,104.599659,112.803301,96.027009,104.734505,105.137550,111.278386,105.583592,...,105.283030,107.194510,96.777127,107.452169,106.418046,101.493387,115.879512,117.195358,106.566308,109.720022
7,CS7,Cal,99.733854,89.315565,114.185174,95.134090,97.928627,102.461321,109.952674,97.619708,...,97.468832,88.705877,92.107681,100.589081,91.646576,96.186802,88.850296,112.136652,93.199687,105.483471


,Name,Type,PFBA_Recovery_%,4:2FTS_Recovery_%,6:2FTS_Recovery_%,8:2FTS_Recovery_%,HFPO-DA_Recovery_%,ADONA_Recovery_%,11Cl-PF3OUdS_Recovery_%,9Cl-PF3ONS_Recovery_%,...,PFHxS_Recovery_%,PFHpS_Recovery_%,PFOS_Recovery_%,PFOSA_Recovery_%,NMeFOSA_Recovery_%,NEtFOSA_Recovery_%,NMeFOSAA_Recovery_%,NEtFOSAA_Recovery_%,PFNS_Recovery_%,PFDS_Recovery_%


ValueError: operands could not be broadcast together with shapes (0,) (7,) 

#### Plotting cal line

In [ ]:
plot_calculated_vs_expected(conc_target_analytes_cal, copy_concs, analyte = "PFOA")

NameError: name 'copy_concs' is not defined

### 2.2.3 RSE results target analytes
We make two results, one with all calibration levels (7 for general compounds, 6 for FTS) - rse_results_cal_target, and one with the calibration level as excluded earlier for EIS compounds - rse_results_excluded_cal.

In [ ]:
%%capture
rse_results_cal_target, rse_results_excluded_cal = calculate_target_rse(conc_target_analytes_cal, copy_concs, p=2, level_to_exclude = excluded_cal_level )

NameError: name 'copy_concs' is not defined

In [ ]:
df_rse_target=df_rse_results(rse_results_cal_target)
display(df_rse_target)

NameError: name 'rse_results_cal_target' is not defined

In [ ]:
df_rse_target_excluded = df_rse_results(rse_results_excluded_cal) 
display(df_rse_target_excluded)

,Compound,RSE_Percent,Calibration_Points,Relationship_Type,Parameters,Pass_Fail,Sum_Relative_Squared_Error
0,NMeFOSE,61.973937,6,quadratic,3,FAIL,0.384077
1,NEtFOSE,62.826841,6,quadratic,3,FAIL,0.394721
2,PFBA,57.214972,6,quadratic,3,FAIL,0.327355
3,HFPO-DA,58.841677,6,quadratic,3,FAIL,0.346234
4,PFPeA,58.000618,6,quadratic,3,FAIL,0.336407
5,4:2FTS,78.997005,5,quadratic,3,FAIL,0.624053
6,6:2FTS,86.955218,5,quadratic,3,FAIL,0.756121
7,8:2FTS,64.404984,5,quadratic,3,FAIL,0.414800
8,NMeFOSAA,69.076801,6,quadratic,3,FAIL,0.477160
9,NEtFOSAA,64.685572,6,quadratic,3,FAIL,0.418422


## 3 Samples 

### 3.1 Recovery EIS in Samples 
Here we compute the recovery of the EIS compounds within the samples analyzed. 

In [ ]:
# computation of EIS concentration and recovery rate
from solutions import NIS_stock, EIS_stock
from allowed_recoveries import EIS_NIS_compound_recovery_aqueous

DF_sample = 1 #dilution factor 
mass_bottle_filled = 499.8 #grams
mass_bottle_empty = 53.68 #grams
Ws = (mass_bottle_filled - mass_bottle_empty)/1000 #L
#Ws_cal = 0.25 #ml

# Create weights dictionary in grams
#soils
weights_dict = {
    'Sand500': 5.0,
    'Sand504': 5.04, 
    'Sand506': 5.06, 
    'Clay537': 5.37
}

#water 23-09
weights_dict = {
    'SPE_Immanuel_50ml_1': (62.45-13.33)/1000,#541.52-56.63,
    'SPE_Immanuel_50ml_2': (62-13.24)/1000, # 519.73-56.00, 
    'SPE_Immanuel_50ml_3': (63.34-13.4)/1000, #541.08-57.60, 
    'SPE_Immanuel_50ml_4': (62.48-13.32)/1000, #539.58-53.62,
    'Alex_SPE50ml_Blank':49.3/1000, # 529.43-57.80
}
rawdata_copy['Ws'] = rawdata_copy['Name'].map(weights_dict)

dilution_NIS_stock = 2#1:1 dilution
#dilution_NIS_stock_cal = 40 #dilution cal
#dilution_EIS_stock_cal = 40 #dilution cal
#spiked_amount_cal = 0.05 #ml
Df_cal = 1 #dilution factor
spiked_amount_NIS = 0.01 #ml
#spiked_amount = 0.01 #ml


dilution_EIS_stock = 1
spiked_amount_EIS =0.05 #ml
#look at this! 
calculated_conc_EIS, NIS_added, EIS_added = calculate_conc_EIS(rawdata_copy, eis_RFS, dilution_NIS_stock,EIS_NIS_analogs, DF_sample, NIS_stock, spiked_amount_NIS, spiked_amount_EIS)

#for run on 12-9-2025
expected_concs_EIS_doubled = {key: 2*value for key, value in expected_concs_EIS.items()}
calculated_recoveries_EIS = calculate_recoveries(calculated_conc_EIS, expected_concs_EIS_doubled)

validation_results_samples= validate_recoveries(calculated_recoveries_EIS, EIS_NIS_compound_recovery_aqueous)
summary_samples = summarize_validation_results(validation_results_samples)
print("Validation Summary:")
for key, value in summary_samples['Overall'].items():
    print(f"{key}: {value}")

failed_recoveries_samples = get_failed_recoveries(validation_results_samples)


  - Rinse
  - Rinse
  - Rinse
  - Alex_SPE50ml_1
  - Alex_SPE50ml_2
  - Alex_SPE50ml_3
  - Alex_SPE50ml_4
  - Rinse

Calculated concentrations for 1 samples with Ws values

NIS masses added per compound:


,NIS_Compound,Mass_Added_ng
0,13C3-PFBA,5.00
1,13C2-PFHxA,2.50
2,13C4-PFOA,2.50
3,13C5-PFNA,1.25
4,13C2-PFDA,1.25
5,18O2-PFHxS,2.50
6,13C4-PFOS,2.50



EIS masses added per compound:


,EIS_Compound,Mass_Added_ng
0,D7-NMeFOSE,250.0
1,D9-NEtFOSE,250.0
2,13C4-PFBA,100.0
3,13C3-HFPO-DA,100.0
4,13C5-PFPeA,50.0
5,13C2-4:2FTS,50.0
6,13C2-6:2FTS,50.0
7,13C2-8:2FTS,50.0
8,D3-NMeFOSAA,50.0
9,D5-NEtFOSAA,50.0



Concentration results:


,Name,Type,Ws,D3-NMeFOSAA_Conc_Calc,13C2-8:2FTS_Conc_Calc,13C2-4:2FTS_Conc_Calc,13C2-PFTeDA_Conc_Calc,D7-NMeFOSE_Conc_Calc,13C8-PFOSA_Conc_Calc,D5-NEtFOSA_Conc_Calc,...,13C2-PFDoA_Conc_Calc,13C9-PFNA_Conc_Calc,13C5-PFHxA_Conc_Calc,D9-NEtFOSE_Conc_Calc,13C3-HFPO-DA_Conc_Calc,13C3-PFBS_Conc_Calc,13C7-PFUnA_Conc_Calc,13C8-PFOS_Conc_Calc,13C4-PFHpA_Conc_Calc,D5-NEtFOSAA_Conc_Calc
14,Alex_SPE50ml_Blank,Sample,0.0493,7.289641,0.341621,1.6714,6.426623,109.859775,894.285759,44.634364,...,28.780513,51.084816,110.676295,124.741408,97.226942,478.882666,84.857739,88.489135,35.358179,5.663977


,Name,Type,D7-NMeFOSE_Recovery_%,D9-NEtFOSE_Recovery_%,13C4-PFBA_Recovery_%,13C3-HFPO-DA_Recovery_%,13C5-PFPeA_Recovery_%,13C2-4:2FTS_Recovery_%,13C2-6:2FTS_Recovery_%,13C2-8:2FTS_Recovery_%,...,13C3-PFHxS_Recovery_%,13C8-PFOS_Recovery_%,13C8-PFOSA_Recovery_%,D3-NMeFOSA_Recovery_%,D5-NEtFOSA_Recovery_%,13C9-PFNA_Recovery_%,13C6-PFDA_Recovery_%,13C7-PFUnA_Recovery_%,13C2-PFDoA_Recovery_%,13C2-PFTeDA_Recovery_%
14,Alex_SPE50ml_Blank,Sample,219.719551,249.482817,2510.767325,486.134709,2022.971539,16.713996,4.296623,3.416209,...,3099.80074,1769.782705,17885.715185,875.270453,892.687277,2043.392621,9246.340989,3394.309555,1151.220531,257.064927


Validation Summary:
Total_Measurements: 24
Passed: 2
Failed: 21
No_Data: 0
No_Reference: 1
Pass_Rate_%: 8.695652173913043


,Sample_Name,Sample_Type,Compound,Calculated_Recovery_%,Expected_Lower_%,Expected_Upper_%,Status,Compound_Type
0,Alex_SPE50ml_Blank,Sample,D7-NMeFOSE,219.719551,10.0,130.0,Fail,EIS
1,Alex_SPE50ml_Blank,Sample,D9-NEtFOSE,249.482817,10.0,130.0,Fail,EIS
2,Alex_SPE50ml_Blank,Sample,13C4-PFBA,2510.767325,5.0,130.0,Fail,EIS
4,Alex_SPE50ml_Blank,Sample,13C5-PFPeA,2022.971539,40.0,130.0,Fail,EIS
5,Alex_SPE50ml_Blank,Sample,13C2-4:2FTS,16.713996,40.0,200.0,Fail,EIS
6,Alex_SPE50ml_Blank,Sample,13C2-6:2FTS,4.296623,40.0,200.0,Fail,EIS
7,Alex_SPE50ml_Blank,Sample,13C2-8:2FTS,3.416209,40.0,300.0,Fail,EIS
10,Alex_SPE50ml_Blank,Sample,13C5-PFHxA,2213.525902,40.0,130.0,Fail,EIS
11,Alex_SPE50ml_Blank,Sample,13C4-PFHpA,707.163575,40.0,130.0,Fail,EIS
12,Alex_SPE50ml_Blank,Sample,13C8-PFOA,276667.665647,40.0,130.0,Fail,EIS


## 3.2 QC samples
In this section we will check the results of the QC samples and assessing if they meet recovery limits. We have already computed EIS recovery for QC earlier. Here, we also compute the concentration for the target analytes in the QC samples.

In [ ]:
sample_QC = "QC"
QC_level = 5
conc_target_analytes_QC = calculate_conc_targets(rawdata_copy,
                           RF_target_analytes,
                           EIS_stock, dilution_EIS_stock_cal, spiked_amount_cal, Ws_cal,
                           Df_cal, sample_QC, debug_compound=None )

calculated_recoveries_EIS_QC = calculate_recoveries(conc_QC, expected_concs_EIS)
expected_concs_level =  filter_expected_concs_by_level(expected_concs_target_analytes,QC_level)
calculated_recoveries_target_QC = calculate_recoveries(conc_target_analytes_QC, expected_concs_level)

validation_results_target_QC= validate_recoveries(calculated_recoveries_target_QC, OPR_recovery_solid)

summary_cal = summarize_validation_results(validation_results_cal)
print("Validation Summary:")
for key, value in summary_cal['Overall'].items():
    print(f"{key}: {value}")

failed_recoveries_cal = get_failed_recoveries(validation_results_cal)


TypeError: 'NoneType' object is not subscriptable